In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
dataset = dgl.data.CoraGraphDataset()
dataset.num_classes

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


7

In [3]:
!ls ~/.dgl/cora_v2

cora_v2_dgl_graph.bin  ind.cora_v2.ally        ind.cora_v2.tx  ind.cora_v2.y
cora_v2_dgl_graph.pkl  ind.cora_v2.graph       ind.cora_v2.ty
ind.cora_v2.allx       ind.cora_v2.test.index  ind.cora_v2.x


In [4]:
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [5]:
# ndata stands for "node data"
g.ndata

{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False])}

In [6]:
# edata stands for "edge data"
g.edata

{}

In [7]:
g.ndata["feat"]

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]])

In [8]:
g.ndata["feat"].shape

torch.Size([2708, 1433])

In [9]:
g.ndata["label"].shape

torch.Size([2708])

In [10]:
g.ndata["label"].unique()

tensor([0, 1, 2, 3, 4, 5, 6])

In [11]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def  __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model w/ given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [12]:
model.parameters

<bound method Module.parameters of GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)>

In [13]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for epoch in range(100):
        # Forward
        logits = model.forward(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # N.B. only compute the loss of the training nodes
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0:
            print(f"(epoch {epoch:03d}) loss = {loss:.3f}, val_acc = {val_acc:.3f} (best = {best_val_acc:.3f}), test_acc = {test_acc:.3f} (best = {best_test_acc:.3f})")

In [14]:
torch.__version__

'1.9.0'

In [15]:
train(g, model)

DGLError: [18:11:38] /opt/dgl/src/array/cpu/./spmm_blocking_libxsmm.h:267: Failed to generate libxsmm kernel for the SpMM operation!
Stack trace:
  [bt] (0) /home/phunc20/.config/miniconda3/envs/dgl/lib/python3.7/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x7fbfa5473e3f]
  [bt] (1) /home/phunc20/.config/miniconda3/envs/dgl/lib/python3.7/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMRedopCsrOpt<long, float, dgl::aten::cpu::op::CopyLhs<float>, dgl::aten::cpu::op::Add<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray)+0x3d4) [0x7fbfa55f36e4]
  [bt] (2) /home/phunc20/.config/miniconda3/envs/dgl/lib/python3.7/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMSumCsrLibxsmm<long, float, dgl::aten::cpu::op::CopyLhs<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runt

- <https://github.com/hfp/libxsmm>